In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def tf_dot(u, v):
    return tf.matmul(tf.transpose(v), u)

def tf_logistic(x):
    ex = tf.exp(x)
    return tf.div(ex, tf.add(1.0, ex))

In [3]:
N = 150 # number of observations
M = 4 # number of parameters
X = tf.placeholder("float", [None, M]) # create symbolic variables
Y = tf.placeholder("float", [None, 1.0])

betas = tf.Variable(tf.random_normal([M, 1], stddev=1.0))
eta = tf.matmul(X, betas)
p = tf_logistic(eta)

In [4]:
negative_log_likelihood = tf.neg(
    tf.add(
        tf_dot( tf.log(p), Y),
        tf_dot( tf.sub(1.0, Y), tf.log(tf.sub(1.0, p)))
))
fit_operation = tf.train.GradientDescentOptimizer(0.05).minimize(negative_log_likelihood, var_list=[betas]) 
#fit_operation = tf.train.AdadeltaOptimizer(epsilon=1e-20).minimize(negative_log_likelihood, var_list=[betas]) 

In [5]:
iris = np.genfromtxt("iris.csv", delimiter=",", dtype="float", skip_header=1)
xs = iris[:, 0:4]
ys = iris[:, 4]
ys.shape = (ys.shape[0], 1)

According to R, the coefficients should be:

    Sepal.Length  Sepal.Width Petal.Length  Petal.Width 
        0.749045    -1.894611     1.057191    -2.907072

In [6]:
## this is the only one which sort of works
## both in the sense that it is somewhat consistent, and very roughly agrees with R1
with tf.Session() as sess:
    tf.initialize_all_variables().run()
  #  something like sgd
    for i in np.random.randint(0, N, 1000):
        sess.run(fit_operation, feed_dict={
                X: xs[i,].reshape([1, M]),
                Y: ys[i,].reshape([1, 1])
            })
    print sess.run(betas)

[[ 0.49083239]
 [-1.48677492]
 [ 0.65768701]
 [-2.19421148]]


In [7]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
  # all at once
    sess.run(fit_operation, feed_dict={
                X: xs,
                Y: ys})
    print sess.run(betas)

[[-28.02988434]
 [-15.63471699]
 [-16.0518589 ]
 [ -6.04919624]]


In [8]:
batch_size = 20
with tf.Session() as sess:
    tf.initialize_all_variables().run()
  #  SGD but in batches
    for i in np.random.randint(0, N-batch_size, 1000):
        j = i + batch_size
        sess.run(fit_operation, feed_dict={
                X: xs[i:j,].reshape([batch_size, M]),
                Y: ys[i:j,].reshape([batch_size, 1])
            })
    print sess.run(betas)

[[ nan]
 [ nan]
 [ nan]
 [ nan]]


In [9]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
  # looping through each row individually
    for i in range(N):
        sess.run(fit_operation, feed_dict={
                X: xs[i,].reshape([1, M]),
                Y: ys[i,].reshape([1, 1])
            })
    print sess.run(betas)

[[ 0.15666875]
 [ 1.3942256 ]
 [-1.74518979]
 [-0.53887975]]


In [10]:
## try the exponential decayed learning rate
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.1
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)

fit_operation_decay = tf.train.GradientDescentOptimizer(learning_rate).minimize(negative_log_likelihood, 
                                                                                var_list=[betas],
                                                                                global_step=global_step) 

# doesn't seem to work well

with tf.Session() as sess:
    tf.initialize_all_variables().run()
  #  something like sgd
    for i in np.random.randint(0, N, 200):
        sess.run(fit_operation, feed_dict={
                X: xs[i,].reshape([1, M]),
                Y: ys[i,].reshape([1, 1])
            })
    print sess.run(betas)

[[-1.1413008 ]
 [ 0.27244043]
 [ 0.92517197]
 [-0.80603081]]
